<img align="center" src="img/course.png" width="800">

# 16720 (B)  Object Tracking in Videos - Assignment 6
    Instructor: Kris                          TAs: Wen-Hsuan (Lead), Zen, Yan, Rawal, Paritosh, Qichen

# Instructions

This section should include the visualizations and answers to specifically highlighted questions from Q1 to Q3. This section will need to be uploaded to gradescope as a pdf and manually graded (this is a separate submission from the coding notebooks) 


1. Students are encouraged to work in groups but each student must submit their own work. Include the names of your collaborators in your write up. Code should <span style='color:red'>Not</span>  be shared or copied. Please properly give credits to others by <span style='color:red'>LISTING EVERY COLLABORATOR</span> in the writeup including any code segments that you discussed,  Please <span style='color:red'>DO NOT</span>  use external code unless permitted. Plagiarism is prohibited and may lead to failure of this course.

2.  **Start early!** This homework will take a long time to complete.
    
3. **Questions:** If you have any question, please look at Piazza first and the FAQ page for this homework.

4. All the theory question and manually graded questions should be included in a single writeup (this notebook exported as pdf or a standalone pdf file) and submitted to gradescope: pdf assignment. 

5. **Attempt to verify your implementation as you proceed:** If you don’t verify that your implementation is correct on toy examples, you will risk having a huge issue when you put everything together. We provide some simple checks in the notebook cells, but make sure you verify them on more complicated samples before moving forward. 

6. **Do not import external functions/packages other than the ones already imported in the files:** The current imported functions and packages are enough for you to complete this assignment. If you need to import other functions, please remember to comment them out after submission. Our autograder will crash if you import a new function that the gradescope server does not expect. 

7. Assignments that do not follow this submission rule will be **penalized up to 10\% of the total score**.

# Preliminaries
In this section, we will go through some of the basics of the Lucas-Kanade tracker and the Matthews-Baker tracker. The following table contains a summary of the variables used in the rest of the assignment.

![title](img/variables.png)

## Template
A template describes the object of interest (eg. a car, football) which we wish to track in a video. Traditionally, the tracking algorithm is initialized with a template, which is represented by a bounding box around the object to be tracked in the first frame of the video. For each of the subsequent frames in the video, the tracker will update its estimate of the object in the image. The tracker achieves this by updating its affine warp.

## Warps
What is a warp? An image transformation or warp $\textbf{W}$ is a function that acts on pixel coordinates $\textbf{x} = \left[u\;\;v\right]^T$
and maps pixel values from one place to another in an image $\textbf{x}' =
\left[u'\;\;v'\right]^T$. Simply put, $\textbf{W}$ maps a pixel with coordinates $\textbf{x} = \left[u\;\;v\right]^T$ to $\textbf{x}' =
\left[u'\;\;v'\right]^T$. Translation, rotation, and scaling are all examples of warps. We denote the parameters of the warp function $\textbf{W}$ by $\textbf{p}$: 

$$
\begin{gathered}
\textbf{x}'= \textbf{W}(\textbf{x};\textbf{p})
\end{gathered}
$$

## Affine Warp
An affine warp is a particular kind of warp that can include any combination of translation, scaling, and rotations. An affine warp can be represented by 6 parameters $\textbf{p} = [p_1\;p_2\;p_3\;p_4\;p_5\;p_6]^{T}$. One of the most convenient things about an affine warp is that it is linear; its action on a point with coordinates $\textbf{x} = \left[u\;\;v\right]^T$ can be described as a matrix operation by a $3 \times 3$ matrix $\textbf{W}(\textbf{p})$:,

$$
\begin{gathered}
\left[ \begin{array}{c} u' \\ v' \\ 1 \end{array} \right] = \textbf{W}(\textbf{p}) \left[
\begin{array}{c} u \\ v \\ 1 \end{array} \right]
\end{gathered}
$$

$$
\begin{gathered}
\textbf{W}(\textbf{p}) = \begin{bmatrix} 1+p_1 & p_3 & p_5 \\ p_2 & 1+p_4 & p_6 \\ 0 & 0 & 1 \end{bmatrix}
\end{gathered}
$$

Note: For convenience, when we want to refer to the warp as a function, we will use $\textbf{W}(\textbf{x};\textbf{p})$ and when we want to refer to the matrix for an affine warp, we will use $\textbf{W}(\textbf{p})$. We will use affine warp and affine transformation interchangeably.

# Theory Questions (30 pts)

Before implementing the trackers, let's study some simple problems that will be useful during the implementation first. The answers to the below questions should be relatively short, consisting of a few lines of math and text.  

## Q1.1
Assuming the affine warp model defined above, derive the expression for the $\frac{\partial \textbf{W}}{\partial \textbf{p}}$ in terms of the warp parameters $\textbf{p} = [p_1\;p_2\;p_3\;p_4\;p_5\;p_6]'$.

$$
W(x; p) = 
\begin{pmatrix}
(1+p_1*x) + p_3*y + p_5 \\
p_2*x + (1+p_4)*y + p_6 \\
\end{pmatrix}
=
\begin{pmatrix}
1+p_1 & p_3 & p_5 \\
p_2 & 1+p_4 & p_6 \\
\end{pmatrix}
\begin{pmatrix}
x \\
y \\
1 \\
\end{pmatrix}
$$

$$
\frac{\partial \textbf{W}}{\partial \textbf{p}} = 
\begin{pmatrix}
\frac{\partial W_x}{\partial p_1} & \frac{\partial W_x}{\partial p_2} & \frac{\partial W_x}{\partial p_3} & \frac{\partial W_x}{\partial p_4} & \frac{\partial W_x}{\partial p_5} & \frac{\partial W_x}{\partial p_6}\\
\frac{\partial W_y}{\partial p_1} & \frac{\partial W_y}{\partial p_2} & \frac{\partial W_y}{\partial p_3} & \frac{\partial W_y}{\partial p_4} & \frac{\partial W_y}{\partial p_5} & \frac{\partial W_y}{\partial p_6}\\
\end{pmatrix}
$$

$$
\frac{\partial \textbf{W}}{\partial \textbf{p}} = 
\begin{pmatrix}
x & 0 & y & 0 & 1 & 0 \\
0 & x & 0 & y & 0 & 1 \\
\end{pmatrix}
$$




## Q1.2
Find the computational complexity (Big O notation) for each runtime iteration (computing $\textbf{J}$ and $\textbf{H}^{-1}$) of the Lucas Kanade method. Express your answers in terms of $n$, $m$ and $p$ where $n$ is the number of pixels in the template $\textbf{T}$, $m$ is the number of pixels in an input image $\textbf{I}$ and $p$ is the number of parameters used to describe the warp $W$.

You may refer to the supplementary PDF for more detailed descriptions of the algorithm.

The total computational complexity can be calculated as follows:

1. Warp the pixels in the template image. $O(np)$
2. Calculate the error image. $O(n)$
3. Calculate the gradient of the wraped coordinates. $O(np)$
4. Evaluate Jacobian for each pixels in the template. $O(np)$
5. Calculate $\nabla I \frac{\partial \textbf{W}}{\partial \textbf{p}}$. $O(np)$
6. Calculate the Hessian. $O(np^2)$
7. Calculate the inverse of Hessian. $O(p^3)$
8. Calculate $\sum_x (\nabla I \frac{\partial \textbf{W}}{\partial \textbf{p}})^T (T(x)-I(W(x;p)))$. $O(np)$
9. Multiply the inverse of Hessian and $\sum_x (\nabla I \frac{\partial \textbf{W}}{\partial \textbf{p}})^T (T(x)-I(W(x;p)))$. $O(p^2)$
10. Update p. O(p)

Thus, the total computational comlexity is $O(np^2+p^3)$

## Q1.3
Find the computational complexity (Big O notation) for the initialization step (Precomputing $\textbf{J}$ and $\textbf{H}^{-1}$) and for each runtime iteration of the Matthews-Baker method. Express your answers in terms of $n$, $m$ and $p$ where $n$ is the number of pixels in the template $\textbf{T}$, $m$ is the number of pixels in an input image $\textbf{I}$ and $p$ is the number of parameters used to describe the warp $W$. You may refer to the supplementary PDF for more detailed descriptions of the algorithm.

How does this compare to the run time of the regular Lucas-Kanade method?

The total computational complexity can be calculated as follows:

1. Calculate the gradient of the template image. $O(n)$
2. Calculate the Jacobian. $O(np)$
3. Calculate $\nabla T \frac{\partial \textbf{W}}{\partial \textbf{p}}$. $O(np)$
4. Calculate the Hessian. $O(np^2)$
5. Then, for each iteration:
   1. Warp the pixels in the template image. $O(np)$
   2. Calculate the error image. $O(n)$
   3. Calculate $\sum_x (\nabla I \frac{\partial \textbf{W}}{\partial \textbf{p}})^T (T(x)-I(W(x;p)))$. $O(np)$
   4. Calculate the inverse of Hessian. $O(p^3)$
   5. Multiply the inverse of Hessian and $\sum_x (\nabla I \frac{\partial \textbf{W}}{\partial \textbf{p}})^T (T(x)-I(W(x;p)))$. $O(p^2)$
6. Update the wrap. O(p^2)

Thus, the total computational comlexity is $O(np+p^3)$

From the computational complexity, we can see that the Matthews-Baker method will have faster run time than the Lucas-Kanade method. 

# Coding Questions Write-up

## Q1.1

Code:

``` python
def LucasKanade(It, It1, rect, thresh=.025, maxIt=100):
    
    '''
    Q1.1: Lucas-Kanade Forward Additive Alignment with Translation Only
    
      Inputs: 
        It: template image
        It1: Current image
        rect: Current position of the object
        (top left, bottom right coordinates, x1, y1, x2, y2)
        thresh: Stop condition when dp is too small
        maxIt: Maximum number of iterations to run
        
      Outputs:
        p: movement vector dx, dy
    '''

    # Set thresholds (you probably want to play around with the values)
    p = np.zeros(2) # dx, dy
    threshold = thresh
    maxIters = maxIt
    x1, y1, x2, y2 = rect
    
    # ----- TODO -----
    # YOUR CODE HERE

    rect_xs = np.linspace(x1, x2, int(x2-x1))
    rect_ys = np.linspace(y1, y2, int(y2-y1))
    rect_xx, rect_yy = np.meshgrid(rect_xs, rect_ys)

    h, w = It.shape
    interp_It = RectBivariateSpline(np.arange(h), np.arange(w), It)
    interp_It1 = RectBivariateSpline(np.arange(h), np.arange(w), It1)
    tmp = interp_It.ev(rect_yy, rect_xx)

    delta_p = np.array([1e4]*2)
    cnt = 0

    while ((np.linalg.norm(delta_p) >= threshold) and (cnt <= maxIters)):
      rect_xx_, rect_yy_ = rect_xx + p[0], rect_yy + p[1]
      wraped = interp_It1.ev(rect_yy_, rect_xx_)
      err = tmp - wraped

      Ix = interp_It1.ev(rect_yy_, rect_xx_, 0, 1)
      Iy = interp_It1.ev(rect_yy_, rect_xx_, 1, 0)
      Ix, Iy = Ix.reshape((-1, 1)), Iy.reshape((-1, 1))
      I = np.hstack((Ix, Iy))

      jacb = np.eye(2)
      A = np.dot(I, jacb)
      H = np.dot(A.T, A)
      delta_p = np.dot(np.dot(np.linalg.inv(H), A.T), err.reshape((-1, 1)))
      p[0] += delta_p[0, 0]
      p[1] += delta_p[1, 0]
      cnt += 1
    
    # raise NotImplementedError()

    return p
```

(To reduce file size, reslts are provided every 10 frames)

car1:
 
<img src="./img/Q1_1_car1_0.png">
<img src="./img/Q1_1_car1_1.png">
<img src="./img/Q1_1_car1_2.png">
<img src="./img/Q1_1_car1_3.png">
<img src="./img/Q1_1_car1_4.png">
<img src="./img/Q1_1_car1_5.png">
<img src="./img/Q1_1_car1_6.png">
<img src="./img/Q1_1_car1_7.png">
<img src="./img/Q1_1_car1_8.png">
<img src="./img/Q1_1_car1_9.png">
<img src="./img/Q1_1_car1_10.png">
<img src="./img/Q1_1_car1_11.png">
<img src="./img/Q1_1_car1_12.png">
<img src="./img/Q1_1_car1_13.png">
<img src="./img/Q1_1_car1_14.png">
<img src="./img/Q1_1_car1_15.png">
<img src="./img/Q1_1_car1_16.png">
<img src="./img/Q1_1_car1_17.png">
<img src="./img/Q1_1_car1_18.png">
<img src="./img/Q1_1_car1_19.png">
<img src="./img/Q1_1_car1_20.png">
<img src="./img/Q1_1_car1_21.png">
<img src="./img/Q1_1_car1_22.png">
<img src="./img/Q1_1_car1_23.png">
<img src="./img/Q1_1_car1_24.png">
<img src="./img/Q1_1_car1_25.png">

car2:
 
<img src="./img/Q1_1_car2_0.png">
<img src="./img/Q1_1_car2_1.png">
<img src="./img/Q1_1_car2_2.png">
<img src="./img/Q1_1_car2_3.png">
<img src="./img/Q1_1_car2_4.png">
<img src="./img/Q1_1_car2_5.png">
<img src="./img/Q1_1_car2_6.png">
<img src="./img/Q1_1_car2_7.png">
<img src="./img/Q1_1_car2_8.png">
<img src="./img/Q1_1_car2_9.png">
<img src="./img/Q1_1_car2_10.png">
<img src="./img/Q1_1_car2_11.png">
<img src="./img/Q1_1_car2_12.png">
<img src="./img/Q1_1_car2_13.png">
<img src="./img/Q1_1_car2_14.png">
<img src="./img/Q1_1_car2_15.png">
<img src="./img/Q1_1_car2_16.png">
<img src="./img/Q1_1_car2_17.png">
<img src="./img/Q1_1_car2_18.png">
<img src="./img/Q1_1_car2_19.png">
<img src="./img/Q1_1_car2_20.png">
<img src="./img/Q1_1_car2_21.png">
<img src="./img/Q1_1_car2_22.png">
<img src="./img/Q1_1_car2_23.png">
<img src="./img/Q1_1_car2_24.png">
<img src="./img/Q1_1_car2_25.png">
<img src="./img/Q1_1_car2_26.png">
<img src="./img/Q1_1_car2_27.png">
<img src="./img/Q1_1_car2_28.png">
<img src="./img/Q1_1_car2_29.png">
<img src="./img/Q1_1_car2_30.png">
<img src="./img/Q1_1_car2_31.png">
<img src="./img/Q1_1_car2_32.png">
<img src="./img/Q1_1_car2_33.png">
<img src="./img/Q1_1_car2_34.png">
<img src="./img/Q1_1_car2_35.png">
<img src="./img/Q1_1_car2_36.png">
<img src="./img/Q1_1_car2_37.png">
<img src="./img/Q1_1_car2_38.png">
<img src="./img/Q1_1_car2_39.png">
<img src="./img/Q1_1_car2_40.png">
<img src="./img/Q1_1_car2_41.png">

landing:
 
<img src="./img/Q1_1_landing_0.png">
<img src="./img/Q1_1_landing_1.png">
<img src="./img/Q1_1_landing_2.png">
<img src="./img/Q1_1_landing_3.png">
<img src="./img/Q1_1_landing_4.png">

race:
 
<img src="./img/Q1_1_race_0.png">
<img src="./img/Q1_1_race_1.png">
<img src="./img/Q1_1_race_2.png">
<img src="./img/Q1_1_race_3.png">
<img src="./img/Q1_1_race_4.png">
<img src="./img/Q1_1_race_5.png">
<img src="./img/Q1_1_race_6.png">

ballet:
 
<img src="./img/Q1_1_ballet_0.png">
<img src="./img/Q1_1_ballet_1.png">
<img src="./img/Q1_1_ballet_2.png">
<img src="./img/Q1_1_ballet_3.png">
<img src="./img/Q1_1_ballet_4.png">

## Q1.2

Code:

``` python
def LucasKanadeAffine(It, It1, rect, thresh=.025, maxIt=100):
    '''
    Q1.2: Lucas-Kanade Forward Additive Alignment with Affine MAtrix
    
      Inputs: 
        It: template image
        It1: Current image
        rect: Current position of the object
        (top left, bottom right coordinates, x1, y1, x2, y2)
        thresh: Stop condition when dp is too small
        maxIt: Maximum number of iterations to run
        
      Outputs:
        M: Affine mtarix (2x3)
    '''

    # Set thresholds (you probably want to play around with the values)
    M = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])
    threshold = thresh
    maxIters = maxIt
    x1, y1, x2, y2 = rect
    
    # ----- TODO -----
    # YOUR CODE HERE

    rect_xs = np.linspace(x1, x2, int(x2-x1))
    rect_ys = np.linspace(y1, y2, int(y2-y1))
    rect_xx, rect_yy = np.meshgrid(rect_xs, rect_ys)

    h, w = It.shape
    interp_It = RectBivariateSpline(np.arange(h), np.arange(w), It)
    interp_It1 = RectBivariateSpline(np.arange(h), np.arange(w), It1)
    tmp = interp_It.ev(rect_yy, rect_xx)
    delta_p = np.array([1e4]*6)
    cnt = 0

    p = np.zeros((6, 1))

    while ((np.sqrt(np.sum(delta_p**2)) >= threshold) and (cnt <= maxIters)):
      rect_xx_ = (1+p[0, 0])*rect_xx+p[1, 0]*rect_yy+p[2, 0]
      rect_yy_ = p[3, 0]*rect_xx+(1+p[4, 0])*rect_yy+p[5, 0]

      wraped = interp_It1.ev(rect_yy_, rect_xx_)
      err = tmp - wraped

      x_fln = rect_xx_.reshape((-1, 1))
      y_fln = rect_yy_.reshape((-1, 1))

      jacb = np.zeros((x_fln.shape[0]*2, 6))
      jacb[::2] = np.hstack((x_fln, np.zeros((x_fln.shape[0], 1)), y_fln, np.zeros((x_fln.shape[0], 1)), np.ones((x_fln.shape[0], 1)), np.zeros((x_fln.shape[0], 1))))
      jacb[1::2] = np.hstack((np.zeros((x_fln.shape[0], 1)), x_fln, np.zeros((x_fln.shape[0], 1)), y_fln, np.zeros((x_fln.shape[0], 1)), np.ones((x_fln.shape[0], 1))))

      Ix = interp_It1.ev(rect_yy_, rect_xx_, 0, 1).reshape((-1, 1))
      Iy = interp_It1.ev(rect_yy_, rect_xx_, 1, 0).reshape((-1, 1))
      I = np.hstack((Ix, Iy))

      A = np.zeros((x_fln.shape[0], 6))
      for i in range(x_fln.shape[0]): A[i, :] = np.dot(I[i, :], jacb[2*i: 2*i+2, :]).reshape((1, -1))
      
      H = np.dot(A.T, A)
      delta_p = np.dot(np.dot(np.linalg.inv(H), A.T), err.reshape((-1, 1)))

      p[0] += delta_p[0, 0]
      p[1] += delta_p[2, 0]
      p[2] += delta_p[4, 0]
      p[3] += delta_p[1, 0]
      p[4] += delta_p[3, 0]
      p[5] += delta_p[5, 0]
      cnt += 1

    M = np.array([[1+p[0], p[1], p[2]], [p[3], 1+p[4], p[5]]]).reshape((2, 3))

    # raise NotImplementedError()
    
    return M
```

(To reduce file size, reslts are provided every 10 frames)

car1:
 
<img src="./img/Q1_2_car1_0.png">
<img src="./img/Q1_2_car1_1.png">
<img src="./img/Q1_2_car1_2.png">
<img src="./img/Q1_2_car1_3.png">
<img src="./img/Q1_2_car1_4.png">
<img src="./img/Q1_2_car1_5.png">
<img src="./img/Q1_2_car1_6.png">
<img src="./img/Q1_2_car1_7.png">
<img src="./img/Q1_2_car1_8.png">
<img src="./img/Q1_2_car1_9.png">
<img src="./img/Q1_2_car1_10.png">
<img src="./img/Q1_2_car1_11.png">
<img src="./img/Q1_2_car1_12.png">
<img src="./img/Q1_2_car1_13.png">
<img src="./img/Q1_2_car1_14.png">
<img src="./img/Q1_2_car1_15.png">
<img src="./img/Q1_2_car1_16.png">
<img src="./img/Q1_2_car1_17.png">
<img src="./img/Q1_2_car1_18.png">
<img src="./img/Q1_2_car1_19.png">
<img src="./img/Q1_2_car1_20.png">
<img src="./img/Q1_2_car1_21.png">
<img src="./img/Q1_2_car1_22.png">
<img src="./img/Q1_2_car1_23.png">
<img src="./img/Q1_2_car1_24.png">
<img src="./img/Q1_2_car1_25.png">

car2:
 
<img src="./img/Q1_2_car2_0.png">
<img src="./img/Q1_2_car2_1.png">
<img src="./img/Q1_2_car2_2.png">
<img src="./img/Q1_2_car2_3.png">
<img src="./img/Q1_2_car2_4.png">
<img src="./img/Q1_2_car2_5.png">
<img src="./img/Q1_2_car2_6.png">
<img src="./img/Q1_2_car2_7.png">
<img src="./img/Q1_2_car2_8.png">
<img src="./img/Q1_2_car2_9.png">
<img src="./img/Q1_2_car2_10.png">
<img src="./img/Q1_2_car2_11.png">
<img src="./img/Q1_2_car2_12.png">
<img src="./img/Q1_2_car2_13.png">
<img src="./img/Q1_2_car2_14.png">
<img src="./img/Q1_2_car2_15.png">
<img src="./img/Q1_2_car2_16.png">
<img src="./img/Q1_2_car2_17.png">
<img src="./img/Q1_2_car2_18.png">
<img src="./img/Q1_2_car2_19.png">
<img src="./img/Q1_2_car2_20.png">
<img src="./img/Q1_2_car2_21.png">
<img src="./img/Q1_2_car2_22.png">
<img src="./img/Q1_2_car2_23.png">
<img src="./img/Q1_2_car2_24.png">
<img src="./img/Q1_2_car2_25.png">
<img src="./img/Q1_2_car2_26.png">
<img src="./img/Q1_2_car2_27.png">
<img src="./img/Q1_2_car2_28.png">
 
landing:
 
<img src="./img/Q1_2_landing_0.png">
<img src="./img/Q1_2_landing_1.png">
<img src="./img/Q1_2_landing_2.png">
<img src="./img/Q1_2_landing_3.png">
<img src="./img/Q1_2_landing_4.png">
 
race:
 
<img src="./img/Q1_2_race_0.png">
<img src="./img/Q1_2_race_1.png">
<img src="./img/Q1_2_race_2.png">
<img src="./img/Q1_2_race_3.png">
<img src="./img/Q1_2_race_4.png">
<img src="./img/Q1_2_race_5.png">
<img src="./img/Q1_2_race_6.png">

ballet:
 
<img src="./img/Q1_2_ballet_0.png">
<img src="./img/Q1_2_ballet_1.png">
<img src="./img/Q1_2_ballet_2.png">
<img src="./img/Q1_2_ballet_3.png">
<img src="./img/Q1_2_ballet_4.png">

## Q2.1

Code:

```python
def InverseCompositionAffine(It, It1, rect, thresh=.025, maxIt=100):
    '''
    Q2.1: Matthew-Bakers Inverse Compositional Alignment with Affine MAtrix
    
      Inputs: 
        It: template image
        It1: Current image
        rect: Current position of the object
        (top left, bottom right coordinates, x1, y1, x2, y2)
        thresh: Stop condition when dp is too small
        maxIt: Maximum number of iterations to run
        
      Outputs:
        M: Affine mtarix (2x3)
    '''
    
    # Set thresholds (you probably want to play around with the values)
    M = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])
    threshold = thresh
    maxIters = maxIt
    x1, y1, x2, y2 = rect
    
    # ----- TODO -----
    # YOUR CODE HERE

    p = np.zeros((6, 1))
    h, w = It.shape

    # put your implementation here
    interp_It = RectBivariateSpline(np.arange(h), np.arange(w), It)
    interp_It1 = RectBivariateSpline(np.arange(h), np.arange(w), It1)

    if ((x1 > x2) or (y1 > y2)): return M 
    # rect_xs = np.linspace(x1, x2+0.1, int(x2+0.1-x1))
    # rect_ys = np.linspace(y1, y2+0.1, int(y2+0.1-y1))
    
    rect_xs = np.arange(x1, x2+0.1)
    rect_ys = np.arange(y1, y2+0.1)
    rect_xx, rect_yy = np.meshgrid(rect_xs, rect_ys)
    tmp = interp_It.ev(rect_yy, rect_xx)

    Tx, Ty = interp_It.ev(rect_yy, rect_xx, 0, 1), interp_It.ev(rect_yy, rect_xx, 1, 0)
    Tx, Ty = Tx.reshape((-1, 1)), Ty.reshape((-1, 1))
    T_grad = np.hstack((Tx, Ty))

    x_flatten, y_flatten = rect_xx.reshape((-1, 1)), rect_yy.reshape((-1, 1))
    jacb = np.zeros((x_flatten.shape[0] * 2, 6))
    jacb[::2] = np.hstack((x_flatten, np.zeros((x_flatten.shape[0], 1)), y_flatten, np.zeros((x_flatten.shape[0], 1)), np.ones((x_flatten.shape[0], 1)), np.zeros((x_flatten.shape[0], 1))))

    jacb[1::2] = np.hstack((np.zeros((x_flatten.shape[0], 1)), x_flatten, np.zeros((x_flatten.shape[0], 1)), y_flatten, np.zeros((x_flatten.shape[0], 1)), np.ones((x_flatten.shape[0], 1))))

    J = np.zeros((x_flatten.shape[0], 6))
    for i in range(x_flatten.shape[0]): J[i, :] = np.dot(T_grad[i, :], jacb[2 * i: 2 * i + 2, :]).reshape((1, -1))

    H = np.dot(J.T, J)

    cnt = 0
    delta_p = np.array([[10000], [10000], [10000], [10000], [10000], [10000]])

    M = np.eye(3)

    while ((np.sqrt(np.sum(delta_p ** 2)) >= threshold) and (cnt <= maxIters)):
      rect_xx_ = M[0, 0] * rect_xx + M[0, 1] * rect_yy + M[0, 2]
      rect_yy_ = M[1, 0] * rect_xx + M[1, 1] * rect_yy + M[1, 2]

      warped = interp_It1.ev(rect_yy_, rect_xx_)

      error = warped - tmp
      delta_p = np.dot(np.dot(np.linalg.inv(H), J.T),
                        error.reshape((-1, 1)))

      original_M = M
      M = np.array([[1.0 + delta_p[0, 0], delta_p[2, 0], delta_p[4, 0]],
                    [delta_p[1, 0], 1.0 + delta_p[3, 0], delta_p[5, 0]],
                    [0.0, 0.0, 1.0]]).reshape(3, 3)

      M = np.dot(original_M, np.linalg.inv(M))
      cnt += 1
    
    #raise NotImplementedError()

    M = M[:2, :]
    
    return M
```

car1:
 
<img src="./img/Q2_1_car1_0.png">
<img src="./img/Q2_1_car1_1.png">
<img src="./img/Q2_1_car1_2.png">
<img src="./img/Q2_1_car1_3.png">
<img src="./img/Q2_1_car1_4.png">
<img src="./img/Q2_1_car1_5.png">
<img src="./img/Q2_1_car1_6.png">
<img src="./img/Q2_1_car1_7.png">
<img src="./img/Q2_1_car1_8.png">
<img src="./img/Q2_1_car1_9.png">
<img src="./img/Q2_1_car1_10.png">
<img src="./img/Q2_1_car1_11.png">
<img src="./img/Q2_1_car1_12.png">
<img src="./img/Q2_1_car1_13.png">
<img src="./img/Q2_1_car1_14.png">
<img src="./img/Q2_1_car1_15.png">
<img src="./img/Q2_1_car1_16.png">
<img src="./img/Q2_1_car1_17.png">
<img src="./img/Q2_1_car1_18.png">
<img src="./img/Q2_1_car1_19.png">
<img src="./img/Q2_1_car1_20.png">
<img src="./img/Q2_1_car1_21.png">
<img src="./img/Q2_1_car1_22.png">
<img src="./img/Q2_1_car1_23.png">
<img src="./img/Q2_1_car1_24.png">
<img src="./img/Q2_1_car1_25.png">
 
car2:
 
<img src="./img/Q2_1_car2_0.png">
<img src="./img/Q2_1_car2_1.png">
<img src="./img/Q2_1_car2_2.png">
<img src="./img/Q2_1_car2_3.png">
<img src="./img/Q2_1_car2_4.png">
<img src="./img/Q2_1_car2_5.png">
<img src="./img/Q2_1_car2_6.png">
<img src="./img/Q2_1_car2_7.png">
<img src="./img/Q2_1_car2_8.png">
<img src="./img/Q2_1_car2_9.png">
<img src="./img/Q2_1_car2_10.png">
<img src="./img/Q2_1_car2_11.png">
<img src="./img/Q2_1_car2_12.png">
<img src="./img/Q2_1_car2_13.png">
<img src="./img/Q2_1_car2_14.png">
<img src="./img/Q2_1_car2_15.png">
<img src="./img/Q2_1_car2_16.png">
<img src="./img/Q2_1_car2_17.png">
<img src="./img/Q2_1_car2_18.png">
<img src="./img/Q2_1_car2_19.png">
<img src="./img/Q2_1_car2_20.png">
<img src="./img/Q2_1_car2_21.png">
<img src="./img/Q2_1_car2_22.png">
<img src="./img/Q2_1_car2_23.png">
<img src="./img/Q2_1_car2_24.png">
<img src="./img/Q2_1_car2_25.png">
<img src="./img/Q2_1_car2_26.png">
<img src="./img/Q2_1_car2_27.png">
<img src="./img/Q2_1_car2_28.png">
<img src="./img/Q2_1_car2_29.png">
<img src="./img/Q2_1_car2_30.png">
<img src="./img/Q2_1_car2_31.png">
<img src="./img/Q2_1_car2_32.png">
<img src="./img/Q2_1_car2_33.png">
<img src="./img/Q2_1_car2_34.png">
<img src="./img/Q2_1_car2_35.png">
<img src="./img/Q2_1_car2_36.png">
<img src="./img/Q2_1_car2_37.png">
<img src="./img/Q2_1_car2_38.png">
<img src="./img/Q2_1_car2_39.png">
<img src="./img/Q2_1_car2_40.png">
<img src="./img/Q2_1_car2_41.png">
 
landing:
 
<img src="./img/Q2_1_landing_0.png">
<img src="./img/Q2_1_landing_1.png">
<img src="./img/Q2_1_landing_2.png">
<img src="./img/Q2_1_landing_3.png">
<img src="./img/Q2_1_landing_4.png">
 
race:
 
<img src="./img/Q2_1_race_0.png">
<img src="./img/Q2_1_race_1.png">
<img src="./img/Q2_1_race_2.png">
<img src="./img/Q2_1_race_3.png">
<img src="./img/Q2_1_race_4.png">
<img src="./img/Q2_1_race_5.png">
<img src="./img/Q2_1_race_6.png">
 
ballet:
 
<img src="./img/Q2_1_ballet_0.png">
<img src="./img/Q2_1_ballet_1.png">
<img src="./img/Q2_1_ballet_2.png">
<img src="./img/Q2_1_ballet_3.png">
<img src="./img/Q2_1_ballet_4.png">

Clarification:

For questions as follows:

"Compare the results of the Matthew-Bakers Tracker with the previous algorithms you have implemented. How do your algorithms perform on each video? What are the differences of the three algorithms in terms of performance and why do we have those differences?  At what point does the algorithm break down and why does this happen?"

As the questions are the same with Q2.2, answers will be provided in Q2.2.

## Q2.2

All three algorithms worked fine on the 'landing' and 'race' video. 

The Lucas-Kanade Affine and Matthew-Bakers algorithms worked better on the 'landing' video as the size of the bounding box changed.

And due to the fixed size of the bounding box, the Lucas-Kanade Algorithm performed better on the 'car2' video as it won't be affected by the traffic light.

The Matthew-Bakers algorithm performed worse on the 'car2' video as the bounding box goes negative when the car went into shaddows.

All three algorithms performed poorly on the 'ballet' video.

The Lucas-Kanade algorithm is basically accurate. But it can't handle the condition where the object changes in size as the size of the bounding box is fixed.

The Lucas-kanade Affine algorithm is more accurate then the Lucas-Kanade but slower, as the algorithm needs to calculate the Hessian and Jacobian matrix for each iteration.

The Matthew-Bakers algorithm provide similar results with the Lucas-Kanade Affine algorithm but faster, since the Hessian and Jacobian matrix were pre-computed. But the algorithm may fail under certain circumstances. 

## Q3

N/A